In [1]:
# 필요모듈 임포트
import pandas as pd                                    #데이터를 이용시
from dash import Dash, dcc, html, Input, Output, State #웹을 만들때 필요
import plotly.express as px                            #그래프를 만들때 필요
import pymysql                                         #데이터를 연결해주는 역할
from sqlalchemy import create_engine,text              #서버 연결시 필요

In [2]:
##데이터베이스 연결
#접속정보
user = "bigdata"
password = 'Bigdata123!!'
host = '192.168.56.101'
port = '3306'
db = 'SampleDB'

#접속 스크립트
conn_script = f'mysql+pymysql://{user}:{password}@{host}:{port}/{db}'
#connection instance
engine = create_engine(conn_script)
conn = engine.connect()

In [12]:
country = pd.read_sql('select distinct country from world',conn)
country = country.loc[:,"country"].to_list()
country = list(set(country))  #set 중복 제거

In [13]:
## dash application 구성 (front end)
app = Dash(__name__) 
app.layout = html.Div(    
    [
        html.H4("년도별 인구수 변화"),
        dcc.RadioItems(id="radio-items",
                       options = ['라인그래프','막대그래프','파이그래프'], 
                       value = "라인그래프",
                       inline = True
                       ),
        dcc.Dropdown(id="dropdown-items",
                     options = [{"label":i, "value":i} for i in country]),
        html.Hr(),
        dcc.Graph(id="plot-graph")
    ]
)


In [14]:
## backend
@app.callback(
    Output("plot-graph","figure"),
    Input("radio-items","value"),
)
def update_graph(graph_type):
    df = pd.read_sql('select year, sum(pop) as pop from world group by year',conn)
    if graph_type == "라인그래프":
        fig = px.line(df,x='year',y='pop')
    elif graph_type == "막대그래프":
        fig = px.bar(df,x='year',y='pop')
    else:
        fig = px.pie(df,values="pop",names="year")
    return fig   

In [15]:
## 서버구동

app.run_server(host=host,port=7777)

Dash is running on http://192.168.56.101:7777/

Dash is running on http://192.168.56.101:7777/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://192.168.56.101:7777/ (Press CTRL+C to quit)
192.168.56.1 - - [18/Feb/2022 17:23:51] "GET / HTTP/1.1" 200 -
192.168.56.1 - - [18/Feb/2022 17:23:51] "GET /_dash-layout HTTP/1.1" 200 -
192.168.56.1 - - [18/Feb/2022 17:23:51] "GET /_dash-dependencies HTTP/1.1" 200 -
192.168.56.1 - - [18/Feb/2022 17:23:51] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
192.168.56.1 - - [18/Feb/2022 17:23:51] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
192.168.56.1 - - [18/Feb/2022 17:23:51] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
192.168.56.1 - - [18/Feb/2022 17:23:51] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.56.1 - - [18/Feb/2022 17:23:52] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.56.1 - - [18/Feb/2022 17:23:53] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.56.1 - - [18/Feb/2022 17:23:54] "POST /_dash-update-component HTTP/1.1" 200 -
